In [1]:
import numpy as np
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder,OneHotEncoder,KBinsDiscretizer,MinMaxScaler,StandardScaler,PolynomialFeatures

from sklearn.svm import LinearSVC
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier,LogisticRegression

from sklearn.model_selection import train_test_split,cross_val_score,cross_val_predict

from sklearn.metrics import confusion_matrix

from sklearn.metrics import precision_score,recall_score,f1_score,accuracy_score
from sklearn.metrics import precision_recall_curve,roc_curve

from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest,SelectFromModel,f_classif,chi2

from sklearn.decomposition import KernelPCA,PCA


pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)

## load data

In [2]:
# df_train = pd.read_csv('train.csv',na_values=[' '])

## pre processing

In [3]:

# #display(df_train.NOEXCH.unique())
# #df_train.head()
# #df_test = pd.read_csv('train.csv')

# df_train.loc[df_train.NOEXCH == 1, 'NOEXCH']   = 'X'
# df_train.loc[df_train.NOEXCH == 0, 'NOEXCH']   = np.nan
# df_train.loc[df_train.NOEXCH == '1', 'NOEXCH']   = 'X'
# df_train.loc[df_train.NOEXCH == '0', 'NOEXCH']   = np.nan

# df_train.NOEXCH.unique()

In [4]:


#print(df_train.isna().sum())
#df_train.describe().T

In [5]:
# features1= [] 
# #list(['cat1','cat2','cat3'])

# for i in range(3,25):
#     features1.append('RAMNT_' + str(i) )
# features1.append()
# df_train[features1].corr()


In [6]:
# print(df_train[['RAMNTALL']].isna().sum())
# df_train[['RAMNTALL']].apply(lambda x: x.corr(df_train.TARGET_D))

# #df_train.drop("TARGET_B", axis=1).apply(lambda x: x.corr(df_train.TARGET_B))

In [7]:

# c = df_train.corr().abs()

# s = c.unstack()

# so = s.sort_values(kind="quicksort")

# display(so.shape)

# print(so[-2000:-1])

In [8]:
#df_train.corr()['TARGET_B']
# cols=df_train.columns[(df_train.isna().sum() == 0)]
# cols = cols[cols != 'OSOURCE']
# cols = cols[cols != 'STATE']
# display(cols)

# df_new[cols].apply(lambda x: x.corr(df_new.TARGET_B))

## Pre processing

In [9]:
# imputer = SimpleImputer(strategy = 'most_frequent')
# df_train[['AGE']] = imputer.fit_transform(df_train[['AGE']])



## set X,y

In [10]:
# X = df_train.drop(['TARGET_B','TARGET_D'],axis=1)
# y = df_train.TARGET_B

# X_train,X_test,y_train ,y_test = train_test_split(X,y,random_state = 42,train_size = 0.8,test_size=0.2)
# #X_train,X_test ,y_train ,y_test

# #print(X_train.isna().sum().to_string())

In [11]:
# X_train_sub = X_train[['AGE']]
# X_test_sub = X_test[['AGE']]



## LogisticRegression

In [12]:
# model = LogisticRegression()
# model.fit(X_train_sub,y_train)

# #y_pred = model.predict(X_test_sub)
# y_test_pred = cross_val_predict(model,X_test_sub,y_test,cv=3)
# conf_mat = confusion_matrix(y_test,y_test_pred)
# f1 = f1_score(y_test,y_test_pred)

# print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(model.score(X_test_sub, y_test)))
# print('f1 of logistic regression classifier on test set: {:.2f}'.format(f1))
# display(conf_mat)

## SGDClassifier

In [13]:
# sgd_clf = SGDClassifier(random_state=0,max_iter=50)
# sgd_clf.fit(X_train_sub,y_train)

# #y_pred = model.predict(X_test_sub)
# y_test_pred = cross_val_predict(sgd_clf,X_test_sub,y_test,cv=3)
# conf_mat = confusion_matrix(y_test,y_test_pred)
# f1 = f1_score(y_test,y_test_pred)

# print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(sgd_clf.score(X_test_sub, y_test)))
# print('f1 of logistic regression classifier on test set: {:.2f}'.format(f1))
# display(conf_mat)


### RandomForestClassifier

In [14]:
# forest_clf = RandomForestClassifier(n_estimators=1)
# forest_clf.fit(X_train_sub,y_train)

# #y_pred = model.predict(X_test_sub)
# y_test_pred = cross_val_predict(forest_clf,X_test_sub,y_test,cv=3)
# conf_mat = confusion_matrix(y_test,y_test_pred)
# f1 = f1_score(y_test,y_test_pred)

# print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(forest_clf.score(X_test_sub, y_test)))
# print('f1 of logistic regression classifier on test set: {:.2f}'.format(f1))
# display(conf_mat)

### LogisticRegression + PolynomialFeatures

In [15]:
# poly = PolynomialFeatures(degree=4)

# model = LogisticRegression(solver='lbfgs', multi_class='multinomial')
# model = make_pipeline(poly, model)
# model.fit(X_train_sub, y_train)

# #y_pred = model.predict(X_test_sub)
# y_test_pred = cross_val_predict(model,X_test_sub,y_test,cv=3)
# conf_mat = confusion_matrix(y_test,y_test_pred)
# f1 = f1_score(y_test,y_test_pred)

# print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(model.score(X_test_sub, y_test)))
# print('f1 of logistic regression classifier on test set: {:.2f}'.format(f1))
# display(conf_mat)
    


### SGDClassifier + PolynomialFeatures

In [16]:
# poly = PolynomialFeatures(degree=4)

# sgd_clf = SGDClassifier(random_state=0,max_iter=50)
# sgd_clf = make_pipeline(poly, sgd_clf)
# sgd_clf.fit(X_train_sub, y_train)

# #y_pred = model.predict(X_test_sub)
# y_test_pred = cross_val_predict(sgd_clf,X_test_sub,y_test,cv=3)
# conf_mat = confusion_matrix(y_test,y_test_pred)
# f1 = f1_score(y_test,y_test_pred)

# print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(sgd_clf.score(X_test_sub, y_test)))
# print('f1 of logistic regression classifier on test set: {:.2f}'.format(f1))
# display(conf_mat)
    


### KNeighborsClassifier

In [17]:
# poly = PolynomialFeatures(degree=4)

# knn = make_pipeline(poly, KNeighborsClassifier(n_neighbors=5))
# knn.fit(X_train_sub, y_train)

# #y_pred = model.predict(X_test_sub)
# y_test_pred = cross_val_predict(knn,X_test_sub,y_test,cv=3)
# conf_mat = confusion_matrix(y_test,y_test_pred)
# f1 = f1_score(y_test,y_test_pred)

# print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(knn.score(X_test_sub, y_test)))
# print('f1 of logistic regression classifier on test set: {:.2f}'.format(f1))
# display(conf_mat)

### ColumnTransformer

In [18]:
# #print(X_train.isna().sum())
# # X_train.info()
# # print(X_train.select_dtypes(include=['float64','int64']).columns)
# X = df_train.drop(['TARGET_B','TARGET_D'],axis=1)
# y = df_train.TARGET_B

# print(X.shape)

# numeric_features = X.select_dtypes(include=['float64','int64']).columns
# numeric_transformer = Pipeline([('imputer',SimpleImputer(strategy='median'))])

# #create the preprocessing pipeline for the categorial data
# category_features = X.select_dtypes(include=['object']).columns
# category_transformer = Pipeline([('imputer',SimpleImputer(strategy='most_frequent')),
#                                ('onehot',OneHotEncoder())])


# preprocessor = ColumnTransformer([
#      ('num',numeric_transformer,numeric_features)
#      #('cat',category_transformer,category_features)
# ])#,remainder='passthrough')

# clf = Pipeline([
#     ('pre',preprocessor)
#     #('cls',RandomForestClassifier(n_estimators=5))
# ])



# #X_train,X_test,y_train ,y_test = train_test_split(X,y,test_size=0.2,random_state = 42)
# #X_train,X_test ,y_train ,y_test

# X_train_pre = clf.fit_transform(X)




## VarianceThreshold

In [19]:
### VarianceThreshold


# print(X_train_pre.shape)

# sel = VarianceThreshold(threshold=0.05)
# X_train_pre = sel.fit_transform(X_train_pre)

# print(X_train_pre.shape)







In [20]:
# #X_train_pre_centered = X_train_pre - X_train_pre.mean(axis=0)
# df = pd.DataFrame(X_train_pre)
# df


## train / Test

In [21]:
# X_train,X_test,y_train ,y_test = train_test_split(df,y,test_size=0.2,random_state = 42,stratify=y)
# X_train,X_test ,y_train ,y_test

# print(X_train_pre.shape)
# # print(X_train_pre.info())
# # X_train = df.select_dtypes(include=['float64','int64'])
# #print(X_train)




### PCA & RandomForestClassifier

In [22]:
# pca = PCA(n_components = 20)
# X_train_after_pca = pca.fit_transform(X_train)
# X_test_after_pca = pca.transform(X_test)

# forest_clf = RandomForestClassifier(n_estimators=15)
# forest_clf.fit(X_train_after_pca, y_train)

# #y_pred = model.predict(X_test_sub)
# y_test_pred = cross_val_predict(forest_clf,X_test_after_pca, y_test, cv=3)
# conf_mat = confusion_matrix(y_test,y_test_pred)
# f1 = f1_score(y_test,y_test_pred)
# accuracy = accuracy_score(y_test,y_test_pred)

# print('Accuracy  classifier on test set: {:.2f}'.format(accuracy))
# print('f1 of logistic regression classifier on test set: {:.2f}'.format(f1))
# display(conf_mat)

In [23]:
# clf_selected = make_pipeline(
#         SelectKBest(f_classif, k=15),  LinearSVC()
# )
# clf_selected.fit(X_train, y_train)


# y_test_pred = cross_val_predict(clf_selected,X_test, y_test, cv=3,)
# conf_mat = confusion_matrix(y_test,y_test_pred)

# f1 = f1_score(y_test,y_test_pred)
# accuracy = accuracy_score(y_test,y_test_pred)

# print('Accuracy of classifier on test set: {:.2f}'.format(accuracy))
# print('f1 of logistic regression classifier on test set: {:.2f}'.format(f1))
# display(conf_mat)

#######################

In [24]:

# numeric_features = ['AGE']
# numeric_transformer = Pipeline([('imputer',SimpleImputer(strategy='most_frequent'))])

# category_features = ['STATE','GENDER']
# category_transformer = Pipeline([('imputer',SimpleImputer(strategy='most_frequent')),
#                                ('onehot',OneHotEncoder(sparse=False))])


# preprocessor = ColumnTransformer([
#      ('num',numeric_transformer,numeric_features),
#      ('cat',category_transformer,category_features)
# ])#,remainder='passthrough')

# clf = Pipeline([
#     ('pre',preprocessor)
#     #('cls',RandomForestClassifier(n_estimators=5))
# ])




In [25]:


# df_train = pd.read_csv('train.csv',na_values=[' '])
# #print(df_train.dtypes)

# df_train.loc[df_train.NOEXCH == 1, 'NOEXCH']   = 'X'
# df_train.loc[df_train.NOEXCH == 0, 'NOEXCH']   = np.nan
# df_train.loc[df_train.NOEXCH == '1', 'NOEXCH']   = 'X'
# df_train.loc[df_train.NOEXCH == '0', 'NOEXCH']   = np.nan

# df_train.drop(['OSOURCE'],axis=1,inplace=True)


# X = df_train.drop(['TARGET_B','TARGET_D'],axis=1)
# y = df_train.TARGET_B

# X_train_pre = clf.fit_transform(X)

In [26]:
# X_train,X_test,y_train ,y_test = train_test_split(X_train_pre,y,test_size=0.2,random_state = 42,stratify=y)
# X_train.shape
# #X_train,X_test ,y_train ,y_test

In [27]:
# clf_selected = make_pipeline(
#         SelectKBest(f_classif, k=15),  LinearSVC()
# )
# clf_selected.fit(X_train, y_train)


# y_test_pred = cross_val_predict(clf_selected,X_test, y_test, cv=3,)
# conf_mat = confusion_matrix(y_test,y_test_pred)

# f1 = f1_score(y_test,y_test_pred)
# accuracy = accuracy_score(y_test,y_test_pred)

# print('Accuracy of classifier on test set: {:.2f}'.format(accuracy))
# print('f1 of logistic regression classifier on test set: {:.2f}'.format(f1))
# display(conf_mat)

###########     2    ############

In [28]:
df_train = pd.read_csv('train.csv',na_values=[' '])
df_train_true = df_train[df_train.TARGET_B==1]
df_train_false = df_train[df_train.TARGET_B==0]


C:\Users\DELL\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (6,8,9,10,11,12,52) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [29]:
print(df_train.shape)
print(df_train_true.shape)
print(df_train_false.shape)

(95412, 481)
(4843, 481)
(90569, 481)


In [68]:
# from sklearn.utils import shuffle

# df_train_false = shuffle(df_train_false)

n = df_train_false.shape[0] * 0.5
n = n / df_train_true.shape[0]
n = int(np.round(n,0))
print(n)


df_train_2 = pd.concat([df_train_true] * n )
print(df_train_2.shape)
df_train_2 = pd.concat([df_train, df_train_2])
print(df_train_2.shape)

9
(43587, 481)
(138999, 481)


In [31]:
# corr = df_train_true.corr()

# c1 = corr.abs().unstack()
# s = c1.sort_values(ascending = False)
# t = s[s == 1]
# with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
#     print(t)

In [32]:
# corr_target = df_train.corr()['TARGET_B']

# corr_target[corr_target > 0.5]

In [69]:
df_train = pd.read_csv('train.csv',na_values=[' '])
df_new = df_train_2[['RAMNT_5','MC1','MC2','MC3','TPE10','TPE1','TPE2','TPE3','TPE4','TPE7','TPE9','TPE12','PEC2',
                  'CARDGIFT','NGIFTALL','RAMNTALL','INCOME','AGE']]
#                    'HHAS4',
#                   'HHAS3','HHAS2','HHAS1','IC22','IC21','IC20','IC19','OCC12','LFC1',
#                   'IC18' ,'TPE1', 'TPE13','IC16', 'OCC3' ,'EIC6','EIC3','EIC2','EIC1', 'OCC11','OCC10','OCC8',
#                    'OCC6','OCC2','OCC1','LFC10','LFC9','LFC8','LFC7','LFC6','IC17','IC15', 'LFC4', 'LFC3', 'LFC2']]



clf = Pipeline([
    ('imputer',SimpleImputer(strategy='mean')),
    ('scaler',StandardScaler())
    #('cls',RandomForestClassifier(n_estimators=5))
])


X = df_new
y = df_train_2.TARGET_B

X_pre = clf.fit_transform(X)

X_train,X_test,y_train ,y_test = train_test_split(X_pre,y,test_size=0.2,random_state = 42,stratify=y)

     
clf_selected = make_pipeline(
        SelectKBest(f_classif, k=15),  LinearSVC()
)
clf_selected.fit(X_train, y_train)


y_test_pred = cross_val_predict(clf_selected,X_test, y_test, cv=3,)
conf_mat = confusion_matrix(y_test,y_test_pred)

f1 = f1_score(y_test,y_test_pred)
accuracy = accuracy_score(y_test,y_test_pred)

print('Accuracy of classifier on test set: {:.2f}'.format(accuracy))
print('f1 classifier on test set: {:.2f}'.format(f1))
display(conf_mat)

           
  
                

C:\Users\DELL\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (6,8,9,10,11,12,52) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Accuracy of classifier on test set: 0.65
f1 classifier on test set: 0.07


C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


array([[17743,   371],
       [ 9344,   342]], dtype=int64)